# Datos de Hoteles

En este notebook analizaremos un dataset de reservas hoteleras, realizando modificaciones y adaptaciones para facilitar su estudio y visualización. Exploraremos las características principales de las reservas, los tipos de clientes, los canales de distribución y otros aspectos relevantes para comprender el comportamiento de los huéspedes y las tendencias en el sector hotelero.

In [17]:
!pip install pandas matplotlib seaborn

In [18]:
import pandas as pd
df = pd.read_csv('../data/raw/bookings_hotel.csv')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


Importación de librerias

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

Modificaciones del dataset, adaptado a nuestros requisitos.

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [21]:
import pandas as pd

# Unir columnas y crear la fecha como string
df['arrival_date'] = df['arrival_date_year'].astype(str) + '-' + df['arrival_date_month'] + '-' + df['arrival_date_day_of_month'].astype(str)

# Convertir a datetime
df['arrival_date'] = pd.to_datetime(df['arrival_date'], format='%Y-%B-%d')

# Formatear como string YYYY-MM-DD
df['arrival_date'] = df['arrival_date'].dt.strftime('%Y-%m-%d')


# Calcular el número de dias
df['days_of_reservation'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']


# Eliminar columnas innecesarias
df = df.drop(['agent', 'company', 'arrival_date_week_number', 'arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month', 'lead_time', 'reserved_room_type', 'assigned_room_type',
              'stays_in_weekend_nights', 'stays_in_week_nights', 'booking_changes', 'deposit_type', 'days_in_waiting_list', 'adr', 'total_of_special_requests', 'reservation_status', 'previous_cancellations',
              ], axis=1)


In [22]:
df.head()

,hotel,is_canceled,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_bookings_not_canceled,customer_type,required_car_parking_spaces,reservation_status_date,arrival_date,days_of_reservation
0,Resort Hotel,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,Transient,0,2015-07-01,2015-07-01,0
1,Resort Hotel,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,Transient,0,2015-07-01,2015-07-01,0
2,Resort Hotel,0,1,0.0,0,BB,GBR,Direct,Direct,0,0,Transient,0,2015-07-02,2015-07-01,1
3,Resort Hotel,0,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,Transient,0,2015-07-02,2015-07-01,1
4,Resort Hotel,0,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,Transient,0,2015-07-03,2015-07-01,2


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   adults                          119390 non-null  int64  
 3   children                        119386 non-null  float64
 4   babies                          119390 non-null  int64  
 5   meal                            119390 non-null  object 
 6   country                         118902 non-null  object 
 7   market_segment                  119390 non-null  object 
 8   distribution_channel            119390 non-null  object 
 9   is_repeated_guest               119390 non-null  int64  
 10  previous_bookings_not_canceled  119390 non-null  int64  
 11  customer_type                   119390 non-null  object 
 12  required_car_par

## Descripción de Datos

### 🍽️ Tipos de Régimen Alimenticio (`meal`)

| Código     | Significado       | Descripción                                                                 |
|------------|-------------------|------------------------------------------------------------------------------|
| `BB`       | Bed & Breakfast    | Alojamiento con desayuno incluido.                                          |
| `FB`       | Full Board         | Pensión completa: desayuno, almuerzo y cena incluidos.                      |
| `HB`       | Half Board         | Media pensión: desayuno y una comida adicional (almuerzo o cena).           |
| `SC`       | Self Catering      | Solo alojamiento, sin comidas incluidas.                                    |
| `Undefined`| No definido        | El régimen alimenticio no fue especificado o no se encuentra en la base.    |


### 🧍 Tipos de Cliente (`customer_type`)

| Código             | Descripción                                                                 |
|--------------------|------------------------------------------------------------------------------|
| `Transient`        | Cliente que reserva individualmente sin contrato previo ni condiciones especiales. |
| `Contract`         | Cliente asociado a un contrato corporativo o comercial con el hotel.         |
| `Transient-Party`  | Grupo de clientes transitorios que viajan juntos, como parte de una pequeña excursión o familia. |
| `Group`            | Clientes que forman parte de una reserva de grupo organizada (por ejemplo, tours o eventos). |

### 🛎️ Canal de Distribución (`distribution_channel`)

| Código      | Significado                             | Descripción                                                                 |
|-------------|------------------------------------------|------------------------------------------------------------------------------|
| `Direct`    | Directo                                  | El cliente reservó directamente con el hotel (por teléfono, web propia, etc.). |
| `Corporate` | Corporativo                              | Reserva realizada a través de acuerdos con empresas o agencias corporativas. |
| `TA/TO`     | Travel Agent / Tour Operator             | Reserva gestionada por agencias de viaje o turoperadores.                   |
| `GDS`       | Global Distribution System               | Plataforma global de reservas (como Amadeus, Sabre, Galileo, etc.).         |
| `Undefined` | No definido                              | El canal de distribución no fue especificado en el registro.                |

### 📊 Segmento de Mercado (`market_segment`)

| Código           | Significado                           | Descripción                                                                 |
|------------------|----------------------------------------|------------------------------------------------------------------------------|
| `Direct`         | Directo                                | El cliente contactó y reservó directamente con el hotel.                    |
| `Corporate`      | Corporativo                            | Reservas hechas por empresas o con acuerdos corporativos.                   |
| `Online TA`      | Agencia de viajes online               | Reservas a través de plataformas como Booking.com, Expedia, etc.            |
| `Offline TA/TO`  | Agencia/Turoperador offline            | Reservas hechas mediante agencias tradicionales o turoperadores físicos.    |
| `Complementary`  | Complementaria                         | Estancia gratuita (por ejemplo, cortesía, promociones, empleados, etc.).    |
| `Groups`         | Grupos                                 | Reservas de grupos grandes organizados (tours, eventos, etc.).              |
| `Undefined`      | No definido                            | El segmento no fue especificado o está sin clasificar.                      |
| `Aviation`       | Aviación                               | Reservas vinculadas a aerolíneas (tripulación, convenios, retrasos, etc.).  |



Eliminar reservas que tengan dias = 0

In [24]:
df.drop(df[df['days_of_reservation'] == 0].index, inplace=True)
df.head()

,hotel,is_canceled,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_bookings_not_canceled,customer_type,required_car_parking_spaces,reservation_status_date,arrival_date,days_of_reservation
2,Resort Hotel,0,1,0.0,0,BB,GBR,Direct,Direct,0,0,Transient,0,2015-07-02,2015-07-01,1
3,Resort Hotel,0,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,Transient,0,2015-07-02,2015-07-01,1
4,Resort Hotel,0,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,Transient,0,2015-07-03,2015-07-01,2
5,Resort Hotel,0,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,Transient,0,2015-07-03,2015-07-01,2
6,Resort Hotel,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,Transient,0,2015-07-03,2015-07-01,2


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118675 entries, 2 to 119389
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           118675 non-null  object 
 1   is_canceled                     118675 non-null  int64  
 2   adults                          118675 non-null  int64  
 3   children                        118671 non-null  float64
 4   babies                          118675 non-null  int64  
 5   meal                            118675 non-null  object 
 6   country                         118201 non-null  object 
 7   market_segment                  118675 non-null  object 
 8   distribution_channel            118675 non-null  object 
 9   is_repeated_guest               118675 non-null  int64  
 10  previous_bookings_not_canceled  118675 non-null  int64  
 11  customer_type                   118675 non-null  object 
 12  required_car_parking_

In [26]:
# Renombrar las columnas a español
df.rename(columns={
    'hotel': 'hotel',
    'is_canceled': 'cancelado',
    'adults': 'adultos',
    'children': 'niños',
    'babies': 'bebés',
    'meal': 'régimen',
    'country': 'nacionalidad_húesped',
    'market_segment': 'público_objetivo',
    'distribution_channel': 'canal_distribución',
    'is_repeated_guest': 'cliente_recurrente',
    'previous_bookings_not_canceled': 'reservas_previas_no_canceladas',
    'customer_type': 'tipo_cliente',
    'required_car_parking_spaces': 'plazas_aparcamiento',
    'reservation_status_date': 'fecha_estado_reserva',
    'arrival_date': 'fecha_llegada',
    'days_of_reservation': 'días_reserva'
}, inplace=True)


In [27]:
df.head()

# Guardar el dataframe limpio
df.to_csv('../data/processed/bookings_hotel_clean.csv', index=False)